In [120]:
import pickle
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
from time import time
from sklearn import linear_model
from sklearn import preprocessing
# Library to be able to use bow
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# library to be able to use gnb
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.externals import joblib
import os.path

# Conversor a formato de lectura del DF

In [1]:
data_frame_convert = pd.read_csv("4K_dataset_enron.csv")
address = data_frame_convert['Name_File']
csv_file = "./enronText.csv"

with open(csv_file,'a') as writer:
    h = "Name,Category,Text"
    writer.write(h)
    writer.write('\n')
                
for file in address:
    cat = data_frame_convert[data_frame_convert['Name_File']==file]['Category'].iloc[0]
    if cat == 1:
        path_spam = "./spam/"+file+".txt"
        if os.path.isfile(path_spam):
            
            with open(path_spam, 'r',encoding='utf-8',errors='ignore') as txt_spam:
                text_spam = str(txt_spam.read()).replace('\n','').replace(',','')
            with open(csv_file,'a') as writer:
                    h = file+","+str(cat)+","+text_spam 
                    writer.writelines(h)
                    writer.write('\n')
    
    else:
        path_ham = "./ham/"+file+".txt"
        if os.path.isfile(path_ham):
            
            with open(path_ham, 'r') as txt:
                text_ham = str(txt.read()).replace('\n', '').replace(',','')
            with open(csv_file,'a') as writer:
                    h = file+","+str(cat)+","+text_ham
                    writer.writelines(h)
                    writer.write('\n')
        

NameError: name 'pd' is not defined

# Pipeline predict original

In [46]:
my_tags = ['Ham','Spam']
def predict_pipeline_OVA(pipeline, X_test, y_test):
    y_pred = pipeline.predict(X_test)
    y_test = lb.inverse_transform(y_test)
    my_tags = pipeline.classes_

    evaluate_prediction(y_pred, y_test)
    precision_recall_f1(y_test, y_pred)
    
def evaluate_prediction(predictions, target, title="Confusion matrix"):
    print('accuracy %s' % accuracy_score(target, predictions))
    print(
        'Correctly Classifed {0} out of {1}'.format(accuracy_score(target, predictions, normalize=False), len(target)))
    cm = confusion_matrix(target, predictions)
    print('confusion matrix\n %s' % cm)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plot_confusion_matrix(cm_normalized, title + ' Normalized')
    print('(row=expected, col=predicted)')
    
    # count = 0
    false_predict = predictions == target
    paste_id = test_data['Name'].tolist()
   
    for idx, i in enumerate (false_predict):
        
        if i == False:
            print(paste_id[idx], target[idx], predictions[idx])

def precision_recall_f1(y_test, y_pred):
    print('\nmacro')
    p, r, f, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_pred, average='macro')
    print('Precision: {0:0.3f}, Recall : {1:0.3f}, F1: {2:0.4f}'.format(p, r, f))

    print('\nmicro')
    p, r, f, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_pred, average='micro')
    print('Precision: {0:0.3f}, Recall : {1:0.3f}, F1: {2:0.4f}'.format(p, r, f))

    print('\nweighted')
    p, r, f, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_pred, average='weighted')
    print('Precision: {0:0.3f}, Recall : {1:0.3f}, F1: {2:0.4f}'.format(p, r, f))

    precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred)

    accuracy = np.average(recall)
    
    print('\nprecision: {}'.format(precision))
    print('\nrecall: {}'.format(recall))
    print('\nfscore: {}'.format(fscore))
    print('\nsupport: {}'.format(support))
    print('\nACC:{}'.format(accuracy))
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(my_tags))    
    target_names = my_tags
    plt.xticks(tick_marks, target_names, rotation=90)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def save_classifer(obj, obj_name):
    pickle.dump(obj, open(obj_name, 'wb'))


def load_classifier(obj_name):
    return pickle.load(open(obj_name, 'rb'))

# Entrenamiento del modelo

In [102]:
data_frame = pd.read_csv("enronText.csv")

lb = preprocessing.LabelBinarizer()
train_data, test_data = train_test_split(data_frame, test_size=.3, random_state=9)
    
X = data_frame['Text'].astype(str)
y = data_frame['Category'].astype(str)
y_frame_binary = lb.fit_transform(data_frame['Category'].astype(str))

X_train, X_test, y_train, y_test = train_test_split(X, y_frame_binary, test_size=.3, random_state=9)

In [37]:
def tfidf_lr_builder(train_data):

    vectorizer = TfidfVectorizer(min_df=3, max_features=2945, norm='l2',
                                 use_idf=True, smooth_idf=True, ngram_range=([1, 1]))
    clf = linear_model.LogisticRegression( penalty='l2', C=50,random_state=0)
    tfidf_lr_clf = Pipeline([('vect', vectorizer), ('clf', clf)])
    tfidf_lr_clf = tfidf_lr_clf.fit(train_data['Text'].astype(str), train_data['Category'].astype(str))
    return tfidf_lr_clf

In [125]:
def tfidf_chi_lr_builder(train_data):
    
    
    vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
    features = vectorizer.fit_transform(train_data['Text'])

    chi2_features = chi2(features, train_data['Category']) 
    umbral = chi2_features[1] > 0.85
    count = 0
    final_features = features
    for i, item in enumerate(umbral):
        if item:
            count = count + 1   
    chi2_selector = SelectKBest(chi2, k=count)
    clf = linear_model.LogisticRegression( penalty='l2', C=50,random_state=0)
    tfidf_lr_clf = Pipeline([('vect', chi2_selector), ('clf', clf)])
    tfidf_lr_clf = tfidf_lr_clf.fit(features, train_data['Category'].astype(str))
    return tfidf_lr_clf

In [126]:
tfidf_lr_clf = tfidf_chi_lr_builder(train_data)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [131]:
my_tags = ['Ham','Spam']
def predict_pipeline_OVA(pipeline, X_test, y_test):
    
    #vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
    #features_test = vectorizer.fit_transform(X_test)
    
    y_pred = pipeline.predict(X_test)
    y_test = lb.inverse_transform(y_test)
    my_tags = pipeline.classes_
    evaluate_prediction(y_pred, y_test)
    precision_recall_f1(y_test, y_pred)
    
def evaluate_prediction(predictions, target, title="Confusion matrix"):
    print('accuracy %s' % accuracy_score(target, predictions))
    print(
        'Correctly Classifed {0} out of {1}'.format(accuracy_score(target, predictions, normalize=False), len(target)))
    cm = confusion_matrix(target, predictions)
    print('confusion matrix\n %s' % cm)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plot_confusion_matrix(cm_normalized, title + ' Normalized')
    print('(row=expected, col=predicted)')
    
    # count = 0
    false_predict = predictions == target
    paste_id = test_data['Name'].tolist()
   
    for idx, i in enumerate (false_predict):
        
        if i == False:
            print(paste_id[idx], target[idx], predictions[idx])

def precision_recall_f1(y_test, y_pred):
    print('\nmacro')
    p, r, f, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_pred, average='macro')
    print('Precision: {0:0.3f}, Recall : {1:0.3f}, F1: {2:0.4f}'.format(p, r, f))

    print('\nmicro')
    p, r, f, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_pred, average='micro')
    print('Precision: {0:0.3f}, Recall : {1:0.3f}, F1: {2:0.4f}'.format(p, r, f))

    print('\nweighted')
    p, r, f, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_pred, average='weighted')
    print('Precision: {0:0.3f}, Recall : {1:0.3f}, F1: {2:0.4f}'.format(p, r, f))

    precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred)

    accuracy = np.average(recall)
    
    print('\nprecision: {}'.format(precision))
    print('\nrecall: {}'.format(recall))
    print('\nfscore: {}'.format(fscore))
    print('\nsupport: {}'.format(support))
    print('\nACC:{}'.format(accuracy))
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(my_tags))    
    target_names = my_tags
    plt.xticks(tick_marks, target_names, rotation=90)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def save_classifer(obj, obj_name):
    pickle.dump(obj, open(obj_name, 'wb'))


def load_classifier(obj_name):
    return pickle.load(open(obj_name, 'rb'))

In [132]:
predict_pipeline_OVA(tfidf_lr_clf, test_data['Text'], test_data['Category'])

ValueError: could not convert string to float: 'Subject: re : may wellhead " spot " purchases - requestvance as we discussed yesterday  i will zero the confirmed column in pops for barrett and increase the same for seneca .bobfrom : vance l taylor / enron @ enronxgate on 04 / 26 / 2001 11 : 42 amto : robert cotten / hou / ect @ ectcc : george weissman / hou / ect @ ect  daren j farmer / hou / ect @ enron  melissa graves / enron @ enronxgate  susan smith / hou / ect @ enronsubject : re : may wellhead " spot " purchases - requestbob i \' m still concern about april \' s nom ; if we don \' t take it to zero  than we could have the possibility of the allocations group adding volumes to barrett \' s april deal . this would be incorrect .vlt- - - - - original message - - - - -from : cotten  robertsent : thursday  april 26  2001 9 : 27 amto : taylor  vance l .cc : weissman  george ; daren j farmer / hou / ect @ enron ; graves  melissa ; susan smith / hou / ect @ enronsubject : re : may wellhe

In [36]:
save_classifer(tfidf_lr_clf, 'tfidf_lr_clf.pkl')